In [1]:
!pip install emoji

     |████████████████████████████████| 51kB 127kB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-0.5.3-cp37-none-any.whl size=42176 sha256=8a184866736e1c5048d9ac6f8e701f828c9c6930deadb8afb21317dcf91dade7
  Stored in directory: /Users/simar/Library/Caches/pip/wheels/86/09/26/f944015841423cd516e8a97f30e29be59e53461aea8b7d3458
Successfully built emoji
You should consider upgrading via the 'pip install --upgrade pip' command.


In [154]:
import emoji

In [155]:
#emoji.EMOJI_UNICODE

In [156]:
emoji_dictionary = {
    "0": "\u2764\uFE0F",
    "1": ":baseball:",
    "2": ":grinning_face_with_big_eyes:",
    "3": ":disappointed_face:",
    "4": ":fork_and_knife:"    
}

In [157]:
train = pd.read_csv('emojify/train_emoji.csv', header=None)
test = pd.read_csv('emojify/test_emoji.csv', header=None)
data_train = train.values
data_test = test.values

In [158]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [159]:

for i in range(10):
    print(data_train[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😃
The assignment is too long  😞
I want to go play ⚾


In [160]:
import pandas as pd
import numpy as np

In [161]:
from keras.utils import to_categorical

In [162]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])

print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [163]:
#Creating embeddings to be processed by RNN models

In [165]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [150]:
#convert sentences to vectors - output of embedding layer

In [166]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [167]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

/Users/simar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [168]:
print(XT[0])
print(len(XT[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [169]:
from keras.layers import *
from keras.models import Sequential

In [170]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

W0814 13:10:31.368313 4400731584 deprecation_wrapper.py:119] From /Users/simar/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 13:10:31.441031 4400731584 deprecation_wrapper.py:119] From /Users/simar/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 13:10:31.458173 4400731584 deprecation_wrapper.py:119] From /Users/simar/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 13:10:31.735525 4400731584 deprecation_wrapper.py:119] From /Users/simar/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default inst

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [171]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

W0814 13:11:45.082267 4400731584 deprecation_wrapper.py:119] From /Users/simar/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 13:11:45.113826 4400731584 deprecation_wrapper.py:119] From /Users/simar/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0814 13:11:45.264883 4400731584 deprecation.py:323] From /Users/simar/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 2s 18ms/step - loss: 1.5828 - acc: 0.2542 - val_loss: 1.6084 - val_acc: 0.3571
Epoch 2/40
118/118 [==============================] - 0s 529us/step - loss: 1.5350 - acc: 0.3305 - val_loss: 1.6443 - val_acc: 0.2143
Epoch 3/40
118/118 [==============================] - 0s 573us/step - loss: 1.5025 - acc: 0.2881 - val_loss: 1.6603 - val_acc: 0.2143
Epoch 4/40
118/118 [==============================] - 0s 600us/step - loss: 1.4577 - acc: 0.4153 - val_loss: 1.6158 - val_acc: 0.2857
Epoch 5/40
118/118 [==============================] - 0s 589us/step - loss: 1.4093 - acc: 0.4153 - val_loss: 1.5603 - val_acc: 0.2857
Epoch 6/40
118/118 [==============================] - 0s 708us/step - loss: 1.3554 - acc: 0.4153 - val_loss: 1.4819 - val_acc: 0.3571
Epoch 7/40
118/118 [==============================] - 0s 611us/step - loss: 1.2417 - acc: 0.5593 - val_loss: 1.4361 - val_acc: 0.5000
Epoch 8/40
118/118

In [172]:
model.evaluate(emb_Xt,Yt)

56/56 [==============================] - 0s 257us/step


[0.8115523116929191, 0.7857142857142857]

In [173]:
pred = model.predict_classes(emb_Xt)

In [174]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a very nice raise
😃
😞
she got me a nice present
😃
❤️
ha ha ha it was so funny
😃
😃
he is a good friend
😃
😃
I am upset
😞
😞
We had such a lovely dinner tonight
😃
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious
😞
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😞
Congratulation for having a baby
😃
😃
stop pissing me off
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
😞
Why are you feeling bad
😞
😞
I am upset
😞
😞
give me the ball
⚾
⚾
My grandmother is the love of my life
❤️
❤️
enjoy your game
⚾
⚾
valentine day is near
😃
😃
